In [30]:
import pandas as pd
import numpy as np

In [31]:
header = ['user_id','item_id','rating','timestamp']
df=pd.read_csv('u.data.csv',sep='\t',names=header)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [32]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of items = ' + str(n_items))

Number of users = 943 | Number of items = 1682


In [33]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df,test_size=0.2)

In [34]:
train_data_mat = np.zeros((n_users,n_items))
for line in train_data.itertuples():
    train_data_mat[line[1]-1,line[2]-1] = line[3]

test_data_mat = np.zeros((n_users,n_items))
for line in test_data.itertuples():
    test_data_mat[line[1]-1,line[2]-1] = line[3]

In [35]:
def predict(ratings,similarity,type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:,np.newaxis] )
        pred=mean_user_rating[:,np.newaxis] + similarity.dot(ratings_diff) /np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) /np.array([np.abs(similarity).sum(axis=1)])
        return pred

In [36]:
from sklearn.metrics.pairwise import pairwise_distances
user_sim = pairwise_distances(train_data_mat)
item_sim = pairwise_distances(train_data_mat.T)

In [37]:
item_prediction  = predict(train_data_mat,item_sim,type='item')
user_prediction = predict(train_data_mat,user_sim,type='user')

In [43]:
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np

def rsme(prediction, ground_truth):
    if prediction is None or ground_truth is None:
        raise ValueError("Error: `prediction` or `ground_truth` is None.")
    
    if not isinstance(prediction, np.ndarray) or not isinstance(ground_truth, np.ndarray):
        raise TypeError("Error: `prediction` and `ground_truth` must be NumPy arrays.")

    if prediction.size == 0 or ground_truth.size == 0:
        raise ValueError("Error: `prediction` or `ground_truth` is empty.")

    nonzero_indices = ground_truth.nonzero()
    if len(nonzero_indices[0]) == 0:  # If there are no nonzero values
        raise ValueError("Error: `ground_truth` contains only zeros.")

    prediction = prediction[nonzero_indices].flatten()
    ground_truth = ground_truth[nonzero_indices].flatten()
    
    return sqrt(mean_squared_error(prediction, ground_truth))

# Check before calling rsme
if user_prediction is not None and test_data_mat is not None:
    print('User based CF RSME:', rsme(user_prediction, test_data_mat))
    print('Item based CF RSME:', rsme(item_prediction, test_data_mat))
else:
    print("Error: user_prediction or test_data_mat is None")


Error: user_prediction or test_data_mat is None
